In [2]:
# !wget -O zen_dataset.tar.gz https://www.dropbox.com/s/5ugsinj434yzmu6/zen_dataset.tar.gz?dl=0
# !tar -xzvf zen_dataset.tar.gz

In [3]:
# !pip install catboost

# Ранжирующая модель

Классический пайплайн рекомендаций выглядит следующим образом:

* Отбор кандидатов. Сначала из всей базы документов отбирается некоторое количество кандидатов. Это нужно, потому что в реальном сервисе очень большое количество документов и невозможно на каждый запрос отскорить всё. В качестве "селекторов" могут выступать простые с вычислительной точки зрения модели (например, kNN по эмбеддингам, предсказание FM, SLIM).
* Ранжирующая модель. Отобранные на предыдущем шаге айтемы ранжируются моделью, чтобы отобрать топ документов для пользователя.
* Затем айтемы дополнительно переранжируются с учетом бизнес-логики.

Более подробно этот пайплайн обсуждался на лекции 1.

В этом задании вам предстоит построить и обучить ранжирующую модель на датасете Дзена.

В качестве самой модели применяют модели на основе градиентного бустинга, в данном задании предлагается использовать CatBoost. В качестве фичей модели используем
* Скалярное произведение, косинусное расстояние между пользовательским и айтемным эмбеддингами. Эмбеддинги: *explicit ALS*, *implicit ALS*, обученные в прошлом задании, контентные модели.
* Айтемные и пользовательские статистики (ctr, количество показов и т.п.)

In [4]:
import numpy as np
import pandas as pd

import tqdm
import json
import itertools
import collections

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

sns.set()

In [5]:
item_counts = pd.read_csv('zen_dataset/item_counts.csv', index_col=0)
item_meta = pd.read_csv('zen_dataset/item_meta.gz', compression='gzip', index_col=0)
user_ratings = pd.read_csv('zen_dataset/user_ratings.gz', compression='gzip', index_col=0)

In [6]:
item_counts['itemId'] = item_counts['itemId'].apply(str)
item_meta['itemId'] = item_meta['itemId'].apply(str)

In [7]:
def parse_ratings_history(string):
    return json.loads(string.replace("'", '"'))

In [8]:
user_encoder = LabelEncoder().fit(user_ratings['userId'])
item_encoder = LabelEncoder().fit(item_counts['itemId'])

all_items = item_counts['itemId']
item_indices = item_encoder.transform(all_items)
item_to_id = dict(zip(all_items, item_indices))

In [9]:
all_users = user_ratings['userId']
user_indices = user_encoder.transform(all_users)
user_to_id = dict(zip(all_users, user_indices))

In [10]:
item_meta

,itemId,title,content
0,5480844460835530524,"Нехитрые способы, как самостоятельно проверить...","С раннего детства нам рассказывают, что сама..."
1,25708764690236829,Где находилась сверхсекретная база подводных л...,"Сомневаюсь, что найдётся сейчас человек, котор..."
2,25995859650472943,Тапки ( жуткий рассказ),Год назад эту историю рассказала мне моя родс...
3,26039067597386753,Крутые находки на Aliexpress №1113,"Доброго времени суток, Уважаемые читатели! Доб..."
4,26225874317634871,Нам пообещали высокую инфляцию. Деньги сильно ...,Цены в магазинах и темпы инфляции растут не то...
...,...,...,...
104498,6221825086402198588,Сколько можно заработать на видеокартах с AGP ...,"Очень старые видеокарты с интерфейсом AGP, кот..."
104499,6221897338759013055,Укрытие Роз на зиму,По Вашим просьбам - повторяю пост прошлого год...
104500,6221960724554910431,"Мама, мамочка, мамуля: снимки самого близкого ...",В подборке снимков из нашего архива — фотограф...
104501,6222047264920702976,Что лучше: сдавать наследственную квартиру или...,"""Кварта − новый бренд ПИК-Брокер"" Привет, меня..."


In [11]:
user_ratings

,userId,trainRatings,testRatings
0,-993675863667353526,"{'-5222866277752422391': 0, '-9060464686933784...","{'-2554466548053893601': 0, '-2220576615613681..."
1,-4250619547882954185,"{'-4553947455665416667': 0, '-3876917199970727...","{'-3523829386334236920': 0, '-1723368694207177..."
2,-3847785305345691076,"{'1455441194337562599': 0, '-17536433251064509...","{'1148168316930968740': 0, '706941777097091385..."
3,1785181112918558233,"{'-2853304815794005643': 0, '-8508657381620121...","{'4719491585277936855': 1, '706941777097091385..."
4,-5078748097863903181,"{'-742528302744844176': 1, '-82389094081541106...","{'6294770073358764390': 0, '-40435255593741244..."
...,...,...,...
75905,4954138831959898373,"{'-6764562552262937310': 0, '-7209088773475894...","{'-1685266709911581206': 0, '-5318736529174157..."
75906,4967793435819938014,"{'6602646631687202064': 0, '-38704649420060130...","{'6266145763213971476': 0, '-37217722565891246..."
75907,-7137764184903122777,"{'-7841343107825813783': 0, '-6743959661482653...","{'482864988385991583': 1, '-132806041087172535..."
75908,-2624987805086334956,"{'-8048759438586949657': 0, '-7609637705975070...","{'-5614425467424704155': 0, '62661457632139714..."


In [12]:
item_to_id

{'2260449285691840072': 60017,
 '-8961093042137696748': 49991,
 '-767028345530903101': 41865,
 '4123700977569815347': 71921,
 '992305795279709612': 104463,
 '1834860733252360511': 57262,
 '-3169260100213302766': 13542,
 '6257341665979578135': 85548,
 '-6696326524107237343': 35748,
 '-2804139083268082575': 11261,
 '4458355878827095479': 74059,
 '2566877130360538483': 61919,
 '1917298282597758814': 57846,
 '-7741337029588977132': 42282,
 '4526832577525638176': 74522,
 '-7538463560532528205': 41106,
 '1635227885606818850': 56042,
 '-5562796636693068423': 28615,
 '1897930483633208292': 57702,
 '-8969392676802303423': 50050,
 '1671392159575150039': 56280,
 '3124835814070251061': 65512,
 '7367930623847216988': 92541,
 '3691142168130108512': 69142,
 '8578408429386316534': 100007,
 '1052807535611095123': 52446,
 '-2864145094965641060': 11658,
 '3788168674044752175': 69761,
 '6994220872145181845': 90196,
 '6751242164615561667': 88679,
 '-60615137820477312': 31775,
 '-1329860704125058323': 2096,

In [13]:
user_ratings
rows = []
for i in user_ratings.index:
    user_id = user_to_id[user_ratings.at[i, 'userId']]
    ratings = parse_ratings_history(user_ratings.at[i, 'trainRatings'])
    for item_id, rating in ratings.items():
        rows.append({'userId': user_id, 'itemId': item_to_id[item_id], 'rating': rating})

ratings_df = pd.DataFrame(rows)
ratings_df

,userId,itemId,rating
0,33745,26350,0
1,33745,50647,0
2,33745,70824,0
3,33745,95984,0
4,33745,43613,0
...,...,...,...
42185498,54593,42542,1
42185499,54593,52037,0
42185500,54593,95071,1
42185501,54593,9850,0


In [14]:
train_ratings = ratings_df

rows = []
for i in user_ratings.index:
    user_id = user_to_id[user_ratings.at[i, 'userId']]
    ratings = parse_ratings_history(user_ratings.at[i, 'testRatings'])
    for item_id, rating in ratings.items():
        rows.append({'userId': user_id, 'itemId': item_to_id[item_id], 'rating': rating})

test_ratings = pd.DataFrame(rows)
test_ratings

,userId,itemId,rating
0,33745,9676,0
1,33745,7697,0
2,33745,44463,0
3,33745,94926,0
4,33745,75224,0
...,...,...,...
9324068,54593,26931,0
9324069,54593,21874,0
9324070,54593,66152,0
9324071,54593,85519,0


In [15]:
from scipy.sparse import csr_matrix
# Получение количества уникальных пользователей и айтемов
n_users = ratings_df['userId'].nunique()
n_items = ratings_df['itemId'].nunique()

max_item_id = int(ratings_df['itemId'].max()) + 1

# Создание разреженной матрицы
user_item_matrix = csr_matrix((ratings_df['rating'], (ratings_df['userId'], ratings_df['itemId'])), shape=(n_users, max_item_id))
user_item_matrix

<75910x104503 sparse matrix of type '<class 'numpy.int64'>'
	with 42185503 stored elements in Compressed Sparse Row format>

## ALS (10 points)

Обучите explicit и implicit ALS.

In [16]:
DIMENSION = 10

In [17]:
from implicit.als import AlternatingLeastSquares

def train_eals(data, dimension=DIMENSION, steps=10):
    model = AlternatingLeastSquares(factors=dimension, regularization=0.01, iterations=steps)
    model.fit(user_item_matrix.T)
    return model.user_factors, model.item_factors

def train_ials(data, dimension=DIMENSION, steps=10, alpha=10):
    model = AlternatingLeastSquares(factors=dimension, regularization=0.01, iterations=steps)
    confidence = data.multiply(alpha).astype('float')
    model.fit(confidence.T)

    return model.user_factors, model.item_factors

In [18]:
eals_user_embeddings, eals_item_embeddings = train_eals(user_item_matrix)
eals_user_embeddings

/Users/tyapkinpetr/PycharmProjects/recsys_v1/venv/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.29028892517089844 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

array([[-1.6719835e-02,  7.5251080e-02,  1.2448716e-01, ...,
         5.2082382e-02, -3.1437341e-02,  1.0180724e-01],
       [ 4.5722025e-03, -2.6299984e-03, -6.5443604e-03, ...,
        -7.7283969e-03,  1.3655156e-02,  3.9240057e-04],
       [-1.0864765e-03,  1.0359716e-03,  9.9292619e-04, ...,
         1.5013173e-03,  1.3472040e-02, -5.1806965e-03],
       ...,
       [-4.1956091e-03, -5.7586533e-04,  7.3610707e-03, ...,
         8.1471326e-03,  8.5378634e-03, -2.0817067e-03],
       [-2.0804924e-03,  2.2616042e-03,  2.3539499e-03, ...,
         2.3924548e-03,  5.9018643e-03, -1.6082416e-03],
       [-3.1540705e-05,  1.8732247e-04,  1.5058144e-03, ...,
         5.9714599e-04,  1.8848029e-03, -6.8217429e-04]], dtype=float32)

In [19]:
ials_user_embeddings, ials_item_embeddings = train_ials(user_item_matrix)
ials_user_embeddings

/Users/tyapkinpetr/PycharmProjects/recsys_v1/venv/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.28218579292297363 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

array([[ 1.0224711 ,  0.08795994, -1.0564485 , ...,  1.1975124 ,
         0.38612875,  0.7817437 ],
       [ 0.12913825, -0.28900203,  0.48009056, ..., -0.3864303 ,
         0.08160443,  0.08744188],
       [-0.02916745, -0.01934863,  0.3036321 , ..., -0.05611994,
         0.0876589 , -0.04396782],
       ...,
       [-0.05356177, -0.04512698,  0.10626599, ...,  0.14243446,
         0.12797746, -0.09404612],
       [ 0.02920093,  0.03367558,  0.06688677, ...,  0.03401768,
         0.02171544,  0.02790272],
       [-0.00569752, -0.03904173,  0.05947788, ...,  0.03797066,
         0.01287464, -0.03509548]], dtype=float32)

## Контентная модель (5 points)

Выберите модель по своему усмотрению и посчитайте эмбединги айтемов для них.

Какую размерность эмбедингов вы хотите взять? Почему?

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer


item_meta.fillna('', inplace=True)
item_meta['combined_text'] = item_meta['title'] + " " + item_meta['content']

vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = vectorizer.fit_transform(item_meta['combined_text'])
content_item_embeddings = tfidf_matrix.toarray()
print("Размерность эмбеддинга:", content_item_embeddings.shape)

Размерность эмбеддинга: (104503, 100)


In [21]:
content_item_embeddings

array([[0.10801783, 0.        , 0.        , ..., 0.07202906, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.16287538, 0.        ,
        0.05628205],
       [0.        , 0.        , 0.04062731, ..., 0.        , 0.0194351 ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15154349, 0.        , 0.06059644, ..., 0.        , 0.08696349,
        0.        ],
       [0.01880211, 0.05958296, 0.        , ..., 0.05641978, 0.        ,
        0.        ]])

Я выбрал размерность эмбедингов 100, считаю что ее должно быть достаточно чтобы модель могла отобразить различия между айтемами, которые представляют собой заголовок и текст. Брал исходя из анализа количества различных слов в текстах, так как это важно для моей модели TfidfVectorizer, возможно стоит взять еще больше но во первых будет дольше считается, так же выбор размерности — это баланс между способностью модели хорошо предсказывать и избежанием излишней размерности, которая может ухудшить интерпретируемость и эффективность и привести к переобучению.

### АЛС шаг от контентной модели (5 points)

Для того чтобы учесть контентные связи между айтемами (важно в случае небольшого количества статистики на айтеме или пользователе), но при этом иметь эмбеды пользователей, можно сделать один шаг АЛС (вычисление пользовательских эмбедов). Далее их нужно будет использовать в финальном ранжировании.

In [22]:
user_ratings

,userId,trainRatings,testRatings
0,-993675863667353526,"{'-5222866277752422391': 0, '-9060464686933784...","{'-2554466548053893601': 0, '-2220576615613681..."
1,-4250619547882954185,"{'-4553947455665416667': 0, '-3876917199970727...","{'-3523829386334236920': 0, '-1723368694207177..."
2,-3847785305345691076,"{'1455441194337562599': 0, '-17536433251064509...","{'1148168316930968740': 0, '706941777097091385..."
3,1785181112918558233,"{'-2853304815794005643': 0, '-8508657381620121...","{'4719491585277936855': 1, '706941777097091385..."
4,-5078748097863903181,"{'-742528302744844176': 1, '-82389094081541106...","{'6294770073358764390': 0, '-40435255593741244..."
...,...,...,...
75905,4954138831959898373,"{'-6764562552262937310': 0, '-7209088773475894...","{'-1685266709911581206': 0, '-5318736529174157..."
75906,4967793435819938014,"{'6602646631687202064': 0, '-38704649420060130...","{'6266145763213971476': 0, '-37217722565891246..."
75907,-7137764184903122777,"{'-7841343107825813783': 0, '-6743959661482653...","{'482864988385991583': 1, '-132806041087172535..."
75908,-2624987805086334956,"{'-8048759438586949657': 0, '-7609637705975070...","{'-5614425467424704155': 0, '62661457632139714..."


In [23]:
rows = []
for i in user_ratings.index:
    user_id = user_to_id[user_ratings.at[i, 'userId']]
    ratings = parse_ratings_history(user_ratings.at[i, 'trainRatings'])
    for item_id, rating in ratings.items():
        rows.append({'userId': user_id, 'itemId': item_to_id[item_id], 'rating': rating})

ratings_df = pd.DataFrame(rows)
ratings_df

,userId,itemId,rating
0,33745,26350,0
1,33745,50647,0
2,33745,70824,0
3,33745,95984,0
4,33745,43613,0
...,...,...,...
42185498,54593,42542,1
42185499,54593,52037,0
42185500,54593,95071,1
42185501,54593,9850,0


In [24]:
train_ratings = ratings_df

rows = []
for i in user_ratings.index:
    user_id = user_to_id[user_ratings.at[i, 'userId']]
    ratings = parse_ratings_history(user_ratings.at[i, 'testRatings'])
    for item_id, rating in ratings.items():
        rows.append({'userId': user_id, 'itemId': item_to_id[item_id], 'rating': rating})

test_ratings = pd.DataFrame(rows)
test_ratings

,userId,itemId,rating
0,33745,9676,0
1,33745,7697,0
2,33745,44463,0
3,33745,94926,0
4,33745,75224,0
...,...,...,...
9324068,54593,26931,0
9324069,54593,21874,0
9324070,54593,66152,0
9324071,54593,85519,0


In [ ]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=100, iterations=1, calculate_training_loss=True)

content_item_embeddings = content_item_embeddings.astype(np.float32)
model.item_factors = content_item_embeddings

model.fit(user_item_matrix.T, show_progress=True)

# content_user_embeddings = model.user_factors
# content_user_embeddings

/Users/tyapkinpetr/PycharmProjects/recsys_v1/venv/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 1.5868537425994873 seconds
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

## Catboost (10 points)

Построим эмбеддинговые признаки пары пользователь-айтем.

In [ ]:
class EmbeddingFeatureGetter:
    def __init__(self, user_embeddings, item_embeddings):
        self.user_embeddings = user_embeddings
        self.item_embeddings = item_embeddings

    def get_features(self, user_id, item_ids):
        """
        * user_id -- индекс пользователя для построения признаков
        * item_ids -- список индексов айтемов
        """
        user_embedding = self.user_embeddings[user_id]

        dot = []
        cos = []

        for item_id in item_ids:
            item_embedding = self.item_embeddings[item_id]

            dot_product = np.dot(user_embedding, item_embedding)
            dot.append(dot_product)

            norm_user = np.linalg.norm(user_embedding)
            norm_item = np.linalg.norm(item_embedding)
            cosine_similarity = dot_product / (norm_user * norm_item) if norm_user != 0 and norm_item != 0 else 0
            cos.append(cosine_similarity)

        return dot, cos

In [ ]:
eals_features_getter = EmbeddingFeatureGetter(eals_user_embeddings, eals_item_embeddings)
ials_features_getter = EmbeddingFeatureGetter(ials_user_embeddings, ials_item_embeddings)
content_features_getter = EmbeddingFeatureGetter(content_user_embeddings, content_item_embeddings)

In [ ]:
item_stats_train = train_ratings.groupby('itemId')['rating'].agg([("clicks", "sum"), ("impressions", "count")])
item_stats_train['ctr'] = pd.DataFrame((item_stats_train['clicks'] / item_stats_train['impressions']) * 100)
item_stats_train

In [ ]:
user_stats_train = train_ratings.groupby('userId').agg(
    total_interactions=('rating', 'count'),
    positive_interactions=('rating', 'sum'),
    average_rating=('rating', 'mean'),
    unique_items=('itemId', pd.Series.nunique)
)
user_stats_train

Построим айтемные и пользовательские признаки.

In [ ]:
full_data_train = pd.merge(train_ratings, user_stats_train, on='userId')
full_data_train = pd.merge(full_data_train, item_stats_train, on='itemId')
full_data_train

In [ ]:
item_stats_test = test_ratings.groupby('itemId')['rating'].agg([("clicks", "sum"), ("impressions", "count")])
item_stats_test['ctr'] = pd.DataFrame((item_stats_test['clicks'] / item_stats_test['impressions']) * 100)
user_stats_test = test_ratings.groupby('userId').agg(
    total_interactions=('rating', 'count'),
    positive_interactions=('rating', 'sum'),
    average_rating=('rating', 'mean'),
    unique_items=('itemId', pd.Series.nunique)
)
full_data_test = pd.merge(test_ratings, user_stats_test, on='userId')
full_data_test = pd.merge(full_data_test, item_stats_test, on='itemId')
full_data_test

In [ ]:
def generate_features_for_model(user_ids, item_ids, feature_getters, item_stats, user_stats):
    """
    Генерирует фичи для списка пользователей и айтемов.

    :param user_ids: Список идентификаторов пользователей.
    :param item_ids: Список идентификаторов айтемов.
    :param feature_getters: Словарь с объектами EmbeddingFeatureGetter для разных типов эмбеддингов.
    :param item_stats: DataFrame с айтемными статистиками.
    :param user_stats: DataFrame с пользовательскими статистиками.
    :return: DataFrame с признаками для модели.
    """
    feature_rows = []

    for user_id in user_ids:
        for item_id in item_ids:
            features = {"user_id": user_id, "item_id": item_id}

            # Добавляем скалярное произведение и косинусное расстояние от разных моделей
            for name, getter in feature_getters.items():
                dot, cos = getter.get_features(user_id, [item_id])
                features[f"{name}_dot"] = dot[0]
                features[f"{name}_cos"] = cos[0]

            # Добавляем айтемные и пользовательские статистики
            if item_id in item_stats.index:
                for col in item_stats.columns:
                    features[f"item_{col}"] = item_stats.loc[item_id, col]
            if user_id in user_stats.index:
                for col in user_stats.columns:
                    features[f"user_{col}"] = user_stats.loc[user_id, col]

            feature_rows.append(features)

    return pd.DataFrame(feature_rows)

feature_getters = {
    "eals": eals_features_getter,
    "ials": ials_features_getter,
    "content": content_features_getter
}

#features_df = generate_features_for_model(user_ids, item_ids, feature_getters, item_stats_train, user_stats_екфшт)

Для построения модели catboost будем пользоваться одноименной [библиотекой](https://catboost.ai/en/docs/concepts/python-reference_catboost). Для обучения модели удобно использовать интерфейс библиотеки, с использованием представления данных `Pool`.

In [ ]:
import catboost

In [ ]:
train_features = full_data_train.drop(columns=['userId', 'itemId', 'rating'])
train_labels = full_data_train['rating']
train_group_ids = full_data_train['userId']

In [ ]:
train_pool = catboost.Pool(train_features, train_labels, group_id=train_group_ids)

In [ ]:
test_features = full_data_test.drop(columns=['userId', 'itemId', 'rating'])
test_labels = full_data_test['rating']
test_group_ids = full_data_test['userId']

In [ ]:
test_pool = catboost.Pool(test_features, test_labels, group_id=test_group_ids)

Обучим саму модель. В [документации](https://catboost.ai/en/docs/references/training-parameters/common#loss_function) есть описание того, как передавать лосс, и какие функции возможны.

Попробуйте обучить на лосс бинарной классификации и ранжирования (второе стоит учить с маленьким числом генерируемых пар на `group_id` во избежание вычислительных трудностей).

In [ ]:
cb_Logloss = catboost.CatBoost(params={
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'verbose': 200,
    'learning_rate': 0.1,
    'depth': 4,
    'iterations': 1000
})

cb_PairLogitPairwise = catboost.CatBoost(params={
    'loss_function': 'PairLogitPairwise',
    'eval_metric': 'AUC',
    'verbose': 200,
    'learning_rate': 0.1,
    'depth': 4,
    'iterations': 1000
})

In [ ]:
cb_Logloss.fit(train_pool, eval_set=test_pool)

In [ ]:
#cb_PairLogitPairwise.fit(train_pool, eval_set=test_pool)

In [ ]:
test_predictions_Logloss = cb_Logloss.predict(test_pool, prediction_type='Class')
test_predictions_PairLogitPairwise = cb_Logloss.predict(test_pool, prediction_type='Class')

Оценим качество

In [ ]:
per_user_predictions = test_predictions_Logloss

In [ ]:
from sklearn.metrics import roc_auc_score

def calculate_per_user_auc(user_ratings, per_user_predictions):
    auc_scores = []
    for user_id in np.unique(user_ratings['user_id']):
        true_labels = user_ratings[user_ratings['user_id'] == user_id]['label']
        user_predictions = per_user_predictions[user_id]
        if len(np.unique(true_labels)) == 2:
            auc = roc_auc_score(true_labels, user_predictions)
            auc_scores.append(auc)
    return auc_scores

In [ ]:
per_user_auc = calculate_per_user_auc(user_ratings, per_user_predictions)
print(np.mean(per_user_auc))

**Выводы:**
Включение разнообразных фичей, таких как скалярное произведение и косинусное расстояние между эмбеддингами пользователей и айтемов, а также айтемные и пользовательские статистики, позволяет модели получить более полное представление о взаимосвязи между пользователями и айтемами. Это способствует повышению точности и релевантности рекомендаций.
Насчет различий между Logloss и PairLogitPairwise -- Pairwise более адаптирован для задач ранжирования, где важно учесть структуру взаимоотношений между разными айтемами – например, какие товары предпочтительнее для конкретного пользователя, а не только факт наличия интереса. Logloss применяется в задачах бинарной классификации. В контексте рекомендательных систем ее использование подразумевает моделирование вероятности интереса пользователя к определенному айтему (например, кликнет ли пользователь на товар или нет). Такой подход подразумевает независимое рассмотрение каждого взаимодействия пользователя и айтема.